Attempting to Determine a Threshold for Similarity between 10Qs and a sample legal document 

In [1]:
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from nltk import tokenize
import seaborn
from sklearn import metrics
from preprocessing.legal_filter import file_to_sents, get_legal_bank


ModuleNotFoundError: No module named 'preprocessing'

### Download Universal Sentence Encoder 

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)


In [ ]:
logging.set_verbosity(logging.ERROR)
legal_vectors = model(get_legal_bank())

In [ ]:
# get the sections associates with one particular 10Q
company_code = "ORCL"
release_date = "20180831"
comp_dir = os.path.join(os.path.expanduser("resources/itemized"), company_code)

sections = {}
for root, dirs, files in os.walk(comp_dir):
    for file in files: 
        if release_date in file:
            # get the section identifier (i.e. part2, item1, etc)
            identifier = file.split("_")[3].replace(".txt", "")
            
            sections[identifier] = file

In [ ]:
total_sents = 0
sect_score_maxes = {}
for sect_id, sect in sections.items():
    sents = file_to_sents(os.path.join(comp_dir, sect))
    if not len(sents) > 0: 
        continue
    doc_vectors = model(sents)
    total_sents += len(doc_vectors)
    cosine = metrics.pairwise.cosine_similarity(doc_vectors, legal_vectors)    
    score_maxes = np.amax(cosine, axis=1)
    sect_score_maxes[sect_id] = score_maxes
total_sents

# Exploring Cosine Similarity

In [ ]:
## plot histogram 

def by_value(item):
    # for iterating by sorted value (from largest to smallest) to make larger histograms plot behind smaller ones
    return -1 * len(item[1])


for sect_id, score_maxes in sorted(sect_score_maxes.items(), key=by_value):
    if len(score_maxes) < 10: 
        print("continued")
        continue
    plt.hist(score_maxes, label = sect_id, alpha = 0.7)
#     plt.axvline(np.median(score_maxes), color = 'r', label= (sect_id + " mean"))
#     plt.axvline(np.mean(score_maxes), color = 'g')
plt.xticks(np.arange(0.0, 0.9, 0.1))
plt.title("Maximum Cosine Similarity Between Legal Bank and " + company_code + " Sentences")
plt.legend(loc='upper right')
plt.show()




In [ ]:
# %matplotlib ipympl

sns.heatmap(cosine,cmap="YlGnBu")

plt.title("Cosine Similarity Heatmap between Apple 10Q and Set of Cautionary Statements")
plt.ylabel("2018 Apple 10Q")
plt.xlabel("Set of Forward Looking Statments")

plt.show()


In [ ]:
thresh = 0.9
idcs = np.where(np.logical_and(cosine > 0.28, cosine < 0.3))
np.shape(idcs)
idcs
# print(non_empty[6])

In [ ]:
print(non_empty[26])